In [198]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import espn_scraper as espn #https://github.com/andr3w321/espn_scraper

def strzero(num):
    num = str(num)
    if len(num) == 1:
        num = '0'+num
    return(num)

In [199]:
year = 2018
month = 9
day = 12
date = '20190822'
schedURL = 'https://www.espn.com/wnba/scoreboard/_/date/'+str(year)+strzero(month)+strzero(day)

In [200]:
game_ids = []
#years = list(range(1997,2021))
years = [2020]
years.reverse()

In [201]:
years

[2020]

In [202]:
#get all of the game ids ever
for year in years:
    prev_len = len(game_ids)
    scoreboard_urls= espn.get_all_scoreboard_urls('wnba', year) 
    for scoreboard_url in scoreboard_urls:
        data = espn.get_url(scoreboard_url, cached_path="cached_data")
        try:
            for event in data['content']:  
                if event['analytics']['gameId'] not in game_ids:
                    game_ids.append(event['analytics']['gameId'])
        except:
            pass
    print([year,len(game_ids)-prev_len])


[2020, 132]


In [203]:
len(game_ids)

132

In [204]:
#if True:
def get_game_data(GameID):
    #first get all the game info
    data_type = "recap"
    url = espn.get_game_url(data_type, "wnba", GameID)
    data = espn.get_url(url)
    temp = data['gamepackageJSON']
    temp = temp['header']   
    #date
    Date = temp['competitions'][0]['date']
    #winning team
    if temp['competitions'][0]['competitors'][0]['winner']: #check if 1st team won
        WinningTeam = temp['competitions'][0]['competitors'][0]['team']['abbreviation']
    else: #otehrwise 2nd team won
        WinningTeam = temp['competitions'][0]['competitors'][1]['team']['abbreviation']
    #regular season or playoffs?
    gametypes = ['none','preseason','regular season','playoffs']
    gametype_int = temp['season']['type']
    GameType = gametypes[gametype_int]
    
    ########################################
    #now get all the play-by-play
    GameID = str(GameID)
    URL = 'https://www.espn.com/wnba/playbyplay?gameId='+GameID
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #teams
    teams = [x.text for x in soup.find_all('span', class_='abbrev')]
    AwayTeam = teams[0]
    HomeTeam = teams[1]
    teams = {teams[0]:'away',teams[1]:'home'}
    
    #scrape the shot description table
    times = [x.text for x in soup.find_all('td', class_='time-stamp')]
    plays = [x.text for x in soup.find_all('td', class_='game-details')]
    score = [x.text for x in soup.find_all('td', class_='combined-score')]
    
    #get team names from the logo image urls
    logourl = [x.find('img').get('src') for x in soup.find_all('td', class_='logo')]
    team = [re.match(r"https.*/wnba/500/(?P<team>[a-z]+)\.",x).group('team').upper() for x in logourl]
    homeoraway = [teams[x] for x in team]
    
    #convert to df
    shotdescriptions = pd.DataFrame({'Time': times,
                          'EventDescription': plays,
                          'Score': score,
                          'EventTeam': team,
                          'HomeOrAway':homeoraway})
    shotdescriptions['GameID'] = GameID
    shotdescriptions['Date'] = Date
    shotdescriptions['GameType'] = GameType
    shotdescriptions['WinningTeam'] = WinningTeam
    shotdescriptions['AwayTeam'] = AwayTeam
    shotdescriptions['HomeTeam'] = HomeTeam
    
    
    #get shot chart data for away team
    awayshots = soup.find('ul', class_='shots away-team')
    awayshots = awayshots.find_all('li')
    awayshotchart = []
    for row in awayshots:
        coordtext = row.get('style')
        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
        xcoord = match.group('xcoord')
        ycoord = match.group('ycoord')
        ycoord = str(100-float(ycoord))
        row_data = [row.get('class')[0],
                    row.get('data-homeaway'),
                    row.get('data-period'),
                    row.get('data-shooter'),
                    row.get('data-text'),
                    row.get('id'),
                    xcoord,
                    ycoord]
        awayshotchart.append(row_data)
    
    #get shot chart data for home team
    homeshots = soup.find('ul', class_='shots home-team')
    homeshots = homeshots.find_all('li')
    homeshotchart = []
    for row in homeshots:
        coordtext = row.get('style')
        match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
        xcoord = match.group('xcoord')
        ycoord = match.group('ycoord')
        ycoord = str(100-float(ycoord))
        row_data = [row.get('class')[0],
                    row.get('data-homeaway'),
                    row.get('data-period'),
                    row.get('data-shooter'),
                    row.get('data-text'),
                    row.get('id'),
                    xcoord,
                    ycoord]
        homeshotchart.append(row_data)
    
    awayshotchart = pd.DataFrame(awayshotchart)
    homeshotchart = pd.DataFrame(homeshotchart)
    
    #name columns for shot chart data
    awayshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
    homeshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
    
    #join to shot descriptions
    playindex = 0
    for row in range(len(awayshotchart)):
        not_found = True
        while not_found:
            if shotdescriptions.loc[playindex,'EventDescription']==awayshotchart.loc[row,'EventDescription']:
                if shotdescriptions.loc[playindex,'HomeOrAway']=='away':
                    shotdescriptions.loc[playindex,'ShotOutcome']=awayshotchart.loc[row,'ShotOutcome']
                    shotdescriptions.loc[playindex,'Quarter']=awayshotchart.loc[row,'Quarter']
                    shotdescriptions.loc[playindex,'pid']=awayshotchart.loc[row,'pid']
                    shotdescriptions.loc[playindex,'ShotID']=awayshotchart.loc[row,'ShotID']
                    shotdescriptions.loc[playindex,'xcoord']=awayshotchart.loc[row,'xcoord']
                    shotdescriptions.loc[playindex,'ycoord']=awayshotchart.loc[row,'ycoord']
                    not_found = False
            playindex += 1       
    
    playindex = 0
    for row in range(len(homeshotchart)):
        not_found = True
        while not_found:
            if shotdescriptions.loc[playindex,'EventDescription']==homeshotchart.loc[row,'EventDescription']:
                if shotdescriptions.loc[playindex,'HomeOrAway']=='home':
                    shotdescriptions.loc[playindex,'ShotOutcome']=homeshotchart.loc[row,'ShotOutcome']
                    shotdescriptions.loc[playindex,'Quarter']=homeshotchart.loc[row,'Quarter']
                    shotdescriptions.loc[playindex,'pid']=homeshotchart.loc[row,'pid']
                    shotdescriptions.loc[playindex,'ShotID']=homeshotchart.loc[row,'ShotID']
                    shotdescriptions.loc[playindex,'xcoord']=homeshotchart.loc[row,'xcoord']
                    shotdescriptions.loc[playindex,'ycoord']=homeshotchart.loc[row,'ycoord']
                    not_found = False
            playindex += 1      
    
    #get quarter numbers and scores
    quarter = 1
    shotdescriptions.loc[0,'AwayScore'] = 0
    shotdescriptions.loc[0,'HomeScore'] = 0
    for row in range(0,len(shotdescriptions)):
        if re.match('End of Game',shotdescriptions.loc[row,'EventDescription']) :
            quarter -= 1 #make the end of the game the same as the last quarter
        shotdescriptions.loc[row,'Quarter'] = quarter
        match = re.match(("([0-9]+) - ([0-9]+)"),shotdescriptions.loc[row,'Score'])
        shotdescriptions.loc[row,'AwayScore'] = match.group(1)
        shotdescriptions.loc[row,'HomeScore'] = match.group(2)
        if re.match('End',shotdescriptions.loc[row,'EventDescription']) :
            quarter += 1 #if you reach the end of the quarter, go to the next quarter
    
    return shotdescriptions

In [205]:
espn.get_game_url('playbyplay', "wnba", 401244185)
data = espn.get_url(url)

https://www.espn.com/wnba/recap?gameId=401244185&xhr=1


In [209]:
df = pd.DataFrame()
#game_ids_test = [401230835]
game_ids_test = game_ids
for GameID in game_ids_test:
    try:
        temp = get_game_data(GameID)
        df = df.append(temp)
    except:
        print(GameID)

https://www.espn.com/wnba/recap?gameId=401230733&xhr=1
https://www.espn.com/wnba/recap?gameId=401230734&xhr=1
https://www.espn.com/wnba/recap?gameId=401230735&xhr=1
https://www.espn.com/wnba/recap?gameId=401230461&xhr=1
https://www.espn.com/wnba/recap?gameId=401230462&xhr=1
https://www.espn.com/wnba/recap?gameId=401230463&xhr=1
https://www.espn.com/wnba/recap?gameId=401230464&xhr=1
https://www.espn.com/wnba/recap?gameId=401230465&xhr=1
https://www.espn.com/wnba/recap?gameId=401230466&xhr=1
https://www.espn.com/wnba/recap?gameId=401230467&xhr=1
https://www.espn.com/wnba/recap?gameId=401230468&xhr=1
https://www.espn.com/wnba/recap?gameId=401230469&xhr=1
https://www.espn.com/wnba/recap?gameId=401230470&xhr=1
https://www.espn.com/wnba/recap?gameId=401230471&xhr=1
https://www.espn.com/wnba/recap?gameId=401230472&xhr=1
https://www.espn.com/wnba/recap?gameId=401230753&xhr=1
https://www.espn.com/wnba/recap?gameId=401230754&xhr=1
https://www.espn.com/wnba/recap?gameId=401230755&xhr=1
https://ww

In [210]:
df.to_csv('espn-pbp-2020.csv',index=False)

In [208]:
game_ids[0:3]

[401230733, 401230734, 401230735]